In [3]:
import os
import logging
import numpy as np
from datetime import datetime

from pinf.analytical_field import get_analytic_b_field
from pinf.performance_metrics import metrics
from pinf.trainer import NF2Trainer
from pinf.unpack import load_cube

In [4]:
b = get_analytic_b_field(n=1, m=1, l=0.3, psi=np.pi/2, resolution=64, bounds=[-1, 1, -1, 1, 0, 2])

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

base_path = './run'
meta_path = None

bin = 1

height = 64
spatial_norm = 32
b_norm = 100

meta_info = None
dim = 256
positional_encoding = False
use_potential_boundary = True
potential_strides = 1
use_vector_potential = False
lambda_div = 1e-1
lambda_ff = 1e-1
decay_iterations = 25000
device = None
work_directory = None

total_iterations = 50000
batch_size = 10000
log_interval = 100
validation_interval = 1000
num_workers = 4

# init logging
os.makedirs(base_path, exist_ok=True)
log = logging.getLogger()
log.setLevel(logging.INFO)
for hdlr in log.handlers[:]:  # remove all old handlers
    log.removeHandler(hdlr)
log.addHandler(logging.FileHandler("{0}/{1}.log".format(base_path, "info_log")))  # set the new file handler
log.addHandler(logging.StreamHandler())  # set the new console handler

base_path = os.path.join(base_path, 'dim%d_bin%d_pf%s_ld%s_lf%s' % (
        dim, bin, str(use_potential_boundary), lambda_div, lambda_ff))

b_cube = b[:, :, 0, :]

trainer = NF2Trainer(base_path, b_cube, height, spatial_norm, b_norm, 
                     meta_info=meta_info, dim=dim, positional_encoding=positional_encoding, 
                     meta_path=meta_path, use_potential_boundary=use_potential_boundary, 
                     potential_strides=potential_strides, use_vector_potential=use_vector_potential,
                     lambda_div=lambda_div, lambda_ff=lambda_ff, decay_iterations=decay_iterations,
                     device=device, work_directory=work_directory)

Configuration:
dim: 256, lambda_div: 0.100000, lambda_ff: 0.100000, decay_iterations: 25000, potential: True, vector_potential: False, 
Using device: cuda (gpus 1) ['NVIDIA GeForce RTX 3060']
Potential Boundary: 100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


In [6]:
start_time = datetime.now()
trainer.train(total_iterations, batch_size, 
              log_interval=log_interval, validation_interval=validation_interval, 
              num_workers=num_workers)
log.info(f'TOTAL RUNTIME: {datetime.now() - start_time}')

Training:   0%|          | 99/50000 [00:13<1:52:08,  7.42it/s][Iteration 000100/050000] [BC: 0.03243637; Div: 0.00004523; For: 0.00074829] [0:00:13.806674]
Lambda B: 972.747224
LR: 0.000498
Training:   0%|          | 199/50000 [00:27<1:52:14,  7.39it/s][Iteration 000200/050000] [BC: 0.03159330; Div: 0.00057075; For: 0.00402706] [0:00:28.134830]
Lambda B: 946.237161
LR: 0.000495
Training:   1%|          | 299/50000 [00:42<1:51:51,  7.40it/s][Iteration 000300/050000] [BC: 0.03076068; Div: 0.00214445; For: 0.00919786] [0:00:42.469103]
Lambda B: 920.449572
LR: 0.000493
Training:   1%|          | 399/50000 [00:56<1:53:17,  7.30it/s][Iteration 000400/050000] [BC: 0.02959176; Div: 0.00929697; For: 0.02268718] [0:00:56.690517]
Lambda B: 895.364766
LR: 0.000491
Training:   1%|          | 499/50000 [01:10<1:52:02,  7.36it/s][Iteration 000500/050000] [BC: 0.02820952; Div: 0.01205610; For: 0.03572670] [0:01:11.069107]
Lambda B: 870.963590
LR: 0.000489
Training:   1%|          | 599/50000 [01:25<1:

In [7]:
B = load_cube('/home/tensor/workspace/pinn_study/organize/run/dim256_bin1_pfTrue_ld0.1_lf0.1/extrapolation_result.nf2')

In [8]:
metrics(B, b)

Potential Field: 100%|██████████| 6/6 [00:00<00:00, 29.46it/s]
